In [112]:
import os 
import datetime
from pathlib import Path, PurePath, PosixPath
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'C:\\Users\\tsbau\\git\\idp-radio-1'

In [97]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from skimage.transform import resize
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121
from tensorflow.keras.callbacks import TensorBoard
from src.utils.save_model import save_model, model_set
from src.datasets.generator import ImageDataGenerator

In [98]:
# Load labels and split into training and valid

DATASET_FOLDER = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY"))

data = pd.read_csv(DATASET_FOLDER / 'train.csv', index_col=[0])
data = data[~data['Frontal/Lateral'].str.contains("Lateral")]
data_train, data_val = train_test_split(data, test_size=0.2)

In [99]:
# create generators
columns = ['Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema']
train_datagen = ImageDataGenerator(data_train, DATASET_FOLDER, columns)
valid_datagen = ImageDataGenerator(data_val, DATASET_FOLDER, columns)

In [101]:
# create model
base_model = DenseNet121(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction_layer = Dense(len(columns), activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=prediction_layer)


29089792/29084464 [==============================] - 20s 1us/step


AttributeError: 'tuple' object has no attribute 'layer'

In [ ]:

# compile model
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# fit model
num_epochs = 3
STEP_SIZE_TRAIN = len(data_train) // train_datagen.batch_size
STEP_SIZE_VALID = len(data_val) // valid_datagen.batch_size
result = model.fit_generator(generator=train_datagen,
                             steps_per_epoch=STEP_SIZE_TRAIN,
                             validation_data=valid_datagen,
                             validation_steps=STEP_SIZE_VALID,
                             epochs=num_epochs)

In [147]:
train_datagen[1]

data\chexpert\dev/CheXpert-v1.0-small/train/patient57029/study1/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient24221/study2/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient14308/study12/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient35128/study10/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient30612/study2/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient22166/study1/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient13570/study26/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient05636/study3/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient09332/study1/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient44184/study6/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient42454/study9/view1_frontal.jpg
data\chexpert\dev/CheXpert-v1.0-small/train/patient16023/study20/view1_frontal.jpg
data\chexper

([array([[[0.31878638, 0.31878638, 0.31878638],
          [0.29250536, 0.29250536, 0.29250536],
          [0.2912186 , 0.2912186 , 0.2912186 ],
          ...,
          [0.40436198, 0.40436198, 0.40436198],
          [0.35125996, 0.35125996, 0.35125996],
          [0.35700444, 0.35700444, 0.35700444]],
  
         [[0.30686657, 0.30686657, 0.30686657],
          [0.29089691, 0.29089691, 0.29089691],
          [0.28322419, 0.28322419, 0.28322419],
          ...,
          [0.3540805 , 0.3540805 , 0.3540805 ],
          [0.36641965, 0.36641965, 0.36641965],
          [0.36879213, 0.36879213, 0.36879213]],
  
         [[0.29186389, 0.29186389, 0.29186389],
          [0.28232805, 0.28232805, 0.28232805],
          [0.25983456, 0.25983456, 0.25983456],
          ...,
          [0.39172603, 0.39172603, 0.39172603],
          [0.34261834, 0.34261834, 0.34261834],
          [0.36377528, 0.36377528, 0.36377528]],
  
         ...,
  
         [[0.55867226, 0.55867226, 0.55867226],
          [0.5

In [146]:
str(DATASET_FOLDER) + "/" + "CheXpert-v1.0-small/train/patient57029/study1/view1_frontal.jpg"

'data\\chexpert\\dev/CheXpert-v1.0-small/train/patient57029/study1/view1_frontal.jpg'

In [142]:
str(DATASET_FOLDER)

'data\\chexpert\\devCheXpert-v1.0-small\\train\\patient57029\\study1\\view1_frontal.jpg'

In [140]:

import cv2
from skimage.transform import resize
testpath ="data\chexpert\devCheXpert-v1.0-small/train/patient57029/study1/view1_frontal.jpg"
image=cv2.imread("data\\chexpert\\dev\\CheXpert-v1.0-small\\train\\patient57029\\study1\\view1_frontal.jpg", cv2.IMREAD_GRAYSCALE)

In [141]:
print(image)

[[ 85  74  77 ...  89  89  92]
 [ 85  75  74 ...  96  93  98]
 [ 74  75  78 ...  94  96  92]
 ...
 [143 142 141 ... 131 132 126]
 [142 142 142 ... 131 134 128]
 [144 145 145 ... 133 136 131]]
